# Scratchwork

This notebook is only used for trying out ideas.

In [1]:
import algebras as alg
import json
import os
import numpy as np
import itertools as it
# from pprint import pprint
import pprint as pp

# from itertools import combinations, permutations, product
import itertools as it

In [2]:
# Path to this repo
aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')

# Path to a directory containing Algebra definitions in JSON
alg_dir = os.path.join(aa_path, "Algebras")

## Groups for Testing

### D4 - Dihedral Group on 4 Vertices

In [3]:
d4_path = os.path.join(alg_dir, "d4_dihedral_group_on_4_vertices.json")
!cat {d4_path}

{"type": "Group",
 "name": "D_4",
 "description": "Dihedral group on four vertices",
 "element_names": ["e", "r", "r^2", "r^3", "f", "fr", "r^2f", "rf"],
 "alt_elem_names": ["()", "(0 1 2 3)", "(0 2)(1 3)", "(0 3 2 1)",
                    "(0 1)(2 3)", "(1 3)", "(0 3)(1 2)", "(0 2)"],
 "mult_table": [[0, 1, 2, 3, 4, 5, 6, 7],
                [1, 2, 3, 0, 7, 4, 5, 6],
                [2, 3, 0, 1, 6, 7, 4, 5],
                [3, 0, 1, 2, 5, 6, 7, 4],
                [4, 5, 6, 7, 0, 1, 2, 3],
                [5, 6, 7, 4, 3, 0, 1, 2],
                [6, 7, 4, 5, 2, 3, 0, 1],
                [7, 4, 5, 6, 1, 2, 3, 0]]
}

In [4]:
d4 = alg.Group(d4_path)
d4.about()


Group: D_4
Dihedral group on four vertices
Abelian? False
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       r     r^3       4
      2     r^2     r^2       2
      3     r^3       r       4
      4       f       f       2
      5      fr      fr       2
      6    r^2f    r^2f       2
      7      rf      rf       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 0, 7, 4, 5, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 0, 1, 2, 5, 6, 7, 4],
 [4, 5, 6, 7, 0, 1, 2, 3],
 [5, 6, 7, 4, 3, 0, 1, 2],
 [6, 7, 4, 5, 2, 3, 0, 1],
 [7, 4, 5, 6, 1, 2, 3, 0]]


### V4 - Klein 4 Group

In [5]:
v4_path = os.path.join(alg_dir, "v4_klein_4_group.json")
!cat {v4_path}

{"type": "Group",
 "name": "V4",
 "description": "Klein-4 group",
 "element_names": ["e", "h", "v", "r"],
 "mult_table": [[0, 1, 2, 3],
                [1, 0, 3, 2],
                [2, 3, 0, 1],
                [3, 2, 1, 0]]
}


In [6]:
v4 = alg.Group(v4_path)
v4.about()


Group: V4
Klein-4 group
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       h       h       2
      2       v       v       2
      3       r       r       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


In [7]:
z2 = alg.generate_cyclic_group(2)
z2

Group('Z2',
'Autogenerated cyclic group of order 2',
['e', 'a'],
[[0, 1], [1, 0]]) 

In [8]:
z2_x_z2 = z2 * z2
z2_x_z2

Group('Z2_x_Z2',
'Direct product of Z2 & Z2',
['e:e', 'e:a', 'a:e', 'a:a'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) 

In [9]:
z4 = alg.generate_cyclic_group(4)
z4

Group('Z4',
'Autogenerated cyclic group of order 4',
['e', 'a', 'a^2', 'a^3'],
[[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]) 

### Powerset Group on 3 Items

TODO: Look for the paper by Carolyn Bean, "Group operations on the power set", Journal of Undergraduate Mathematics 8 #1 (March 1976), 13-17.

In [10]:
ps3 = alg.generate_powerset_group(3)
ps3.about()


Group: PS3
Autogenerated group on the powerset of 3 elements, with symmetric difference operator
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]


## Finite Rings

"Consider any finite set. Then its power set with respect to symmetric difference (as 1st operation) and set intersection (as 2nd operation) is an example of a finite ring." -- [Quora](https://www.quora.com/What-are-examples-of-finite-rings#:~:text=Matrices%20over%20any%20ring%20make,a%20finite%20ring%20with%20elements.)

## Set Operations

In [11]:
A = {0, 2, 4, 6, 8}
B = {1, 2, 3, 4, 5}

print(f"               Union: {A | B}")
print(f"        Intersection: {A & B}")
print(f"          Difference: {A - B}")
print(f"Symmetric Difference: {A ^ B}")

               Union: {0, 1, 2, 3, 4, 5, 6, 8}
        Intersection: {2, 4}
          Difference: {0, 8, 6}
Symmetric Difference: {0, 1, 3, 5, 6, 8}


## Summarize Proper Subgroups of a Group

In [12]:
ps3_prop_subs = ps3.proper_subgroups()
print(f"\n{ps3.name} has {len(ps3_prop_subs)} proper subgroups.\n")


PS3 has 14 proper subgroups.



In [13]:
iso_sets_of_groups = alg.divide_groups_into_isomorphic_sets(ps3_prop_subs)

In [14]:
[g.name for g in iso_sets_of_groups[0]]

['PS3_subgroup_0',
 'PS3_subgroup_2',
 'PS3_subgroup_3',
 'PS3_subgroup_4',
 'PS3_subgroup_6',
 'PS3_subgroup_11',
 'PS3_subgroup_12']

In [15]:
[g.name for g in iso_sets_of_groups[1]]

['PS3_subgroup_1',
 'PS3_subgroup_5',
 'PS3_subgroup_7',
 'PS3_subgroup_8',
 'PS3_subgroup_9',
 'PS3_subgroup_10',
 'PS3_subgroup_13']

In [16]:
[[g.name for g in iso_set] for iso_set in iso_sets_of_groups]

[['PS3_subgroup_0',
  'PS3_subgroup_2',
  'PS3_subgroup_3',
  'PS3_subgroup_4',
  'PS3_subgroup_6',
  'PS3_subgroup_11',
  'PS3_subgroup_12'],
 ['PS3_subgroup_1',
  'PS3_subgroup_5',
  'PS3_subgroup_7',
  'PS3_subgroup_8',
  'PS3_subgroup_9',
  'PS3_subgroup_10',
  'PS3_subgroup_13']]

In [17]:
[iso_set[0].about() for iso_set in iso_sets_of_groups]


Group: PS3_subgroup_0
Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {1}     {1}       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]

Group: PS3_subgroup_1
Subgroup of: Autogenerated group on the powerset of 3 elements, with symmetric difference operator
Abelian? True
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1  {0, 1}  {0, 1}       2
      2  {0, 2}  {0, 2}       2
      3  {1, 2}  {1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


[None, None]

## Conjugation & Normal Subgroups

### V4

In [18]:
v4_subs = v4.proper_subgroups()
v4_subs

[Group('V4_subgroup_0',
 'Subgroup of: Klein-4 group',
 ['e', 'h'],
 [[0, 1], [1, 0]]) ,
 Group('V4_subgroup_1',
 'Subgroup of: Klein-4 group',
 ['e', 'v'],
 [[0, 1], [1, 0]]) ,
 Group('V4_subgroup_2',
 'Subgroup of: Klein-4 group',
 ['e', 'r'],
 [[0, 1], [1, 0]]) ]

In [19]:
v4.unique_proper_subgroups()

[Group('V4_subgroup_0',
 'Subgroup of: Klein-4 group',
 ['e', 'h'],
 [[0, 1], [1, 0]]) ]

In [20]:
v4.is_normal(v4_subs[0])

True

### D4

In [21]:
d4.about()


Group: D_4
Dihedral group on four vertices
Abelian? False
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       r     r^3       4
      2     r^2     r^2       2
      3     r^3       r       4
      4       f       f       2
      5      fr      fr       2
      6    r^2f    r^2f       2
      7      rf      rf       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 0, 7, 4, 5, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 0, 1, 2, 5, 6, 7, 4],
 [4, 5, 6, 7, 0, 1, 2, 3],
 [5, 6, 7, 4, 3, 0, 1, 2],
 [6, 7, 4, 5, 2, 3, 0, 1],
 [7, 4, 5, 6, 1, 2, 3, 0]]


In [22]:
len(d4.proper_subgroups())

8

In [23]:
d4_subs = d4.unique_proper_subgroups()
d4_subs

[Group('D_4_subgroup_0',
 'Subgroup of: Dihedral group on four vertices',
 ['e', 'r^2', 'f', 'r^2f'],
 [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]) ,
 Group('D_4_subgroup_1',
 'Subgroup of: Dihedral group on four vertices',
 ['e', 'f'],
 [[0, 1], [1, 0]]) ,
 Group('D_4_subgroup_5',
 'Subgroup of: Dihedral group on four vertices',
 ['e', 'r', 'r^2', 'r^3'],
 [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]) ]

The result below matches the comment in the last paragraph of this section:
https://en.wikipedia.org/wiki/Dihedral_group#Properties

In [24]:
[d4.is_normal(sub) for sub in d4_subs]

[True, False, True]

### V4

In [25]:
grp = v4
grp_subs = grp.proper_subgroups()
[grp.is_normal(sub) for sub in grp_subs]

[True, True, True]

### PS3

In [26]:
grp = ps3
grp_unique_subs = grp.unique_proper_subgroups()
[grp.is_normal(sub) for sub in grp_unique_subs]

[True, True]

### WORK IN PROGRESS

In [27]:
subs = grp.proper_subgroups()
iso_subs = alg.divide_groups_into_isomorphic_sets(subs)
elem_name_lists = [[g.element_names for g in iso_sub] for iso_sub in iso_subs]
elem_name_lists

[[['{}', '{1}'],
  ['{}', '{0}'],
  ['{}', '{2}'],
  ['{}', '{1, 2}'],
  ['{}', '{0, 1}'],
  ['{}', '{0, 1, 2}'],
  ['{}', '{0, 2}']],
 [['{}', '{0, 1}', '{0, 2}', '{1, 2}'],
  ['{}', '{0}', '{2}', '{0, 2}'],
  ['{}', '{0}', '{1, 2}', '{0, 1, 2}'],
  ['{}', '{2}', '{0, 1}', '{0, 1, 2}'],
  ['{}', '{0}', '{1}', '{0, 1}'],
  ['{}', '{1}', '{2}', '{1, 2}'],
  ['{}', '{1}', '{0, 2}', '{0, 1, 2}']]]

In [28]:
def about_proper_subgroups2(grp, unique=False, show_elements=True):
    if unique:
        subs = grp.unique_proper_subgroups()
    else:
        subs = grp.proper_subgroups()
    print(f"\nSubgroups of {grp.name}:")
    for sub in subs:
        print(f"\n  {sub.name}:")
        print(f"       order: {sub.order}")
        if show_elements:
            print(f"    elements: {sub.element_names}")
        print(f"    abelian?: {sub.is_abelian()}")
        print(f"     normal?: {grp.is_normal(sub)}")
    return None

In [29]:
def about_proper_subgroups3(grp, unique=False, show_elements=True):
    subs = grp.proper_subgroups()
    if unique:
        iso_subs = alg.divide_groups_into_isomorphic_sets(subs)
        elem_name_lists = [g.element_names for g in iso_subs[0]]
        elem_name_lists[1:]
    print(f"\nSubgroups of {grp.name}:")
    for sub in subs:
        print(f"\n  {sub.name}:")
        print(f"       order: {sub.order}")
        if show_elements:
            print(f"    elements: {sub.element_names}")
        print(f"    abelian?: {sub.is_abelian()}")
        print(f"     normal?: {grp.is_normal(sub)}")
    return None

In [30]:
about_proper_subgroups2(ps3, unique=True)


Subgroups of PS3:

  PS3_subgroup_0:
       order: 2
    elements: ['{}', '{1}']
    abelian?: True
     normal?: True

  PS3_subgroup_1:
       order: 4
    elements: ['{}', '{0, 1}', '{0, 2}', '{1, 2}']
    abelian?: True
     normal?: True
